In [43]:
cd '/home/darkshadow/mycodinghub/fastapi-nosql-scrape/'

/home/darkshadow/mycodinghub/fastapi-nosql-scrape


/home/darkshadow/mycodinghub/fastapi-nosql-scrape/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [44]:
from app import scraper
from requests_html import HTML
from slugify import slugify
import pprint
import re

In [45]:
url = "https://www.amazon.com/Vita-Prep-Explorian-Commercial-Container-Warranty/dp/B0CB8J793R/ref=sr_1_1_sspa?crid=123H40Z2FWPK9&dib=eyJ2IjoiMSJ9.W2cBja_MKochFhWAIGO0YZjg4_HheUF5sxXBWOHqFkEf4bNFBJxnGgEGb5SQvhYiHKB63keSyaOK0Y6wcyUeLlRz9IZiltXsUE9B15a9WxNDQzqz0lcqzM7BY-O1r5KufgyrhVfHUMjqtEwF5Bx_gQ--r67zEUY58e4y-niPr2--VOu51cStJfS66F33L4lvlnKuCZY1F08urjJO8duzAa4J8FbiNDZazz3w9xqdOdFkqBB-Q8c1-HvXRfPrOnHGgiK5DTag1tEQqV_li7tZqufqRzLg_1cY9KRIgy-e4_s.xCp8M7Ws_Zsa8k-JWsfxLRe6I1fth339p0kEVzzWQ3E&dib_tag=se&keywords=vitamix%2Bblender%2Bparts&qid=1731612355&sprefix=vitamix%2Caps%2C256&sr=8-1-spons&sp_csd=d2lkZ2V0TmFtZT1zcF9hdGY&th=1"

In [46]:
s = scraper.Scraper(url=url, endless_scroll=True)

In [47]:
html_str = s.get()

In [48]:
html_obj = HTML(html=html_str)

In [49]:
def extract_element_text(html_obj, element_id):
    el  = html_obj.find(element_id, first=True)
    if not el:
        return ''
    return el.text

In [50]:
price_str = extract_element_text(html_obj, ".a-price")
price_str

'$59.99'

In [51]:
title_str = extract_element_text(html_obj, "#title")
title_str

'For Vitamix Blender Pitcher 64oz,Replace for Vitamix 5200 5000 6300 7500 VM0197 VM0101 VM0100 VM0103 E310 Asy172 Classic/Explorian/Commercial Series etc Container Jar Cup,10 Years Warranty'

In [52]:
def extract_tables(html_obj):
    return html_obj.find("table")

In [53]:
tables = extract_tables(html_obj)

In [54]:
def extract_price_from_string(value: str, regex=r"[\$]{1}[\d,]+\.?\d{0,2}"):
    x = re.findall(regex, value)
    val = None
    if len(x) == 1:
        val = x[0]
    return val

In [55]:
def extract_dataset(tables):
    dataset = {}
    for table in tables:
        for tbody in table.element.getchildren():
            for tr in tbody.getchildren():
                row = []
                for col in tr.getchildren():
                    content = ""
                    try:
                        content = col.text_content()
                    except:
                        pass
                    if content != "":
                        _content = content.strip()
                        row.append(_content)
                if len(row) != 2:
                    continue
                key = row[0]
                value = row[1]
                data = {}
                key = slugify(key)
                if key in dataset:
                    continue
                else:
                    if "$" in value:
                        new_key = key
                        old_key = f'{key}_raw'
                        new_value = value
                        old_value = value
                        dataset[new_key] = new_value
                        dataset[old_key] = old_value
                    else:
                        dataset[key] = value
    return dataset

In [56]:
dataset = extract_dataset(tables)
pprint.pprint(dataset)

{'amazonglobal-shipping': '$73.91',
 'amazonglobal-shipping_raw': '$73.91',
 'asin': '\u200eB0CB8J793R',
 'best-sellers-rank': '#63,332 in Home & Kitchen (See Top 100 in Home & '
                      'Kitchen) \n'
                      '\n'
                      '#60 in Blender Replacement Parts',
 'capacity': '4 Pounds',
 'country-of-origin': '\u200eChina',
 'customer-reviews': '4.2 out of 5 stars  202Reviews',
 'date-first-available': '\u200eJuly 6, 2023',
 'estimated-import-charges': '$543.95',
 'estimated-import-charges_raw': '$543.95',
 'item-weight': '\u200e2.99 pounds',
 'price': '$59.99',
 'price_raw': '$59.99',
 'product-dimensions': '6.29"D x 6.3"W x 11.8"H',
 'total': '$677.85',
 'total_raw': '$677.85'}
